In [9]:
import numpy as np

In [10]:
"""
https://www.crownmelbourne.com.au/getsydmedia/99be12f2-c48b-44dc-b6ba-ab5849f6f723/crown-melbourne-gaming-blackjack-rules_1.pdf?ext=.pdf
https://en.wikipedia.org/wiki/Glossary_of_blackjack_terms
"""

# configurations
NDECKS: int = 8
DECK_PENETRATION: float = 0.25 # typical number of decks dealt out before reshuffle
# NPLAYERS: int = 4 # affects how many cards you may be able to count

In [11]:
"""
We keep basic card face and suit information. 
Later, could implement weird house rules without major modification.
"""
cards = np.array(['2','3','4','5','6','7','8','9','T','J','Q','K','A']) # use char T for 10 
suits = np.array(['D', 'S', 'C', 'H'])
deck = np.array([np.char.add(cards, suit) for suit in suits]).ravel()
shoe = np.repeat(deck, NDECKS)

# We use bytestrings to enable Cython memview of C char[2] dtypes
shoe = shoe.astype('S2', order='C')

cut_idx = min(round((len(deck) * NDECKS) * DECK_PENETRATION), len(deck) * NDECKS)

In [12]:
# we need this to be fast, we will shuffle a lot!
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
%timeit rng.shuffle(shoe)

7.99 µs ± 89.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
# BAD: redundant for 1D array. 
shoe_size = shoe.shape[0]
%timeit shoe[rng.permutation(shoe_size)]

11.7 µs ± 57.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [13]:
# BAD idea to use value mapping after every shuffle
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(cards.astype('S1'), values))
%timeit np.array([dict_map[x[:1]] for x in shoe]) # slicing byte string to get first char
# %timeit np.vectorize(dict_map.get)(shoe)

66.4 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
# may change to use numpy to read-in csv, to avoid pandas dependency
import pandas as pd
strat = pd.read_csv(r"StrategyInput\BasicNoDeviations-4to8Decks-HitSoft17.csv", header=None)

"""
Read in the individual strategy tables. Cast into byte chars and C contiguous arrays. 
"""
hard = strat.iloc[0:18, :].copy().reset_index(drop=True)
hard = hard.values.astype(str)
hrd = hard[1:,1:].astype('S1', order='C')

soft = strat.iloc[18:27, :].copy().reset_index(drop=True)
soft = soft.values.astype(str)
sft = soft[1:,1:].astype('S1', order='C')

splits = strat.iloc[27:38, :].copy().reset_index(drop=True)
splits = splits.values.astype(str)
splts = splits[1:,1:].astype('S1', order='C')

In [15]:
# Read in the counting strategy
count = pd.read_csv(r"StrategyInput\HiLoCount.csv", header=None)
cnt = count.iloc[:,1].values.astype(float, order='C')

In [16]:
dealer_idx = {
    b'2': 0,
    b'3': 1,
    b'4': 2,
    b'5': 3,
    b'6': 4,
    b'7': 5,
    b'8': 6,
    b'9': 7,
    b'T': 8,
    b'J': 8,
    b'Q': 8,
    b'K': 8,
    b'A': 9,
}

In [2]:
import cardstream
print(cardstream.some_func(5))

21
